In [1]:
import tensorflow as tf
import numpy as np

from __future__ import absolute_import, division, print_function

In [2]:
test = np.array([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])
test

array([[[1, 2],
        [3, 4]],

       [[5, 6],
        [7, 8]]])

In [3]:
np.flip(test, 1).ravel()

array([3, 4, 1, 2, 7, 8, 5, 6])

In [4]:
def init_board():
    board = np.zeros((8, 8, 6), dtype=np.int8)
    
    # Pawns
    board[1, :, 0] = -1
    board[6, :, 0] = -1
    
    # Medium
    for j in range(3):
        board[0, [j, 7-j], j+1] = -1
        board[7, [j, 7-j], j+1] = 1
    
    # Queens
    board[0, 3, 4] = -1
    board[7, 3, 4] = 1
    
    # Kings
    board[0, 4, 5] = -1
    board[7, 4, 5] = 1
    
    return board

In [5]:
board =init_board()

In [6]:
import string, re

In [7]:
move = "Nxf5+"
pattern = re.compile("[\Wx]+") 
pattern.sub("", move)

'Nf5'

In [8]:
def parse_move(move):
    
    # Remove capture/check notation (x/+)
    pattern = re.compile("[\Wx]+") 
    move = pattern.sub("", move)    
    
    piece_dict = {"R": 1, "N": 2, "B": 3, "Q": 4, "K": 5}
    
    # Identify piece type
    if move[0].isupper():
        d = piece_dict[move[0]]
        move = move[1:]
    else:
        d = 0
        
    # Check for ambiguity in move
    if len(move) == 3:
        ambig = move[0]
        move = move[1:]
    elif len(move) > 3:
        return -1
    else:
        ambig = None
     
    
    rank_dict = {"a": 7, "b": 6, "c": 5, "d": 4, "e": 3, "f": 2, "g": 1, "h": 0}
    
    r, f = rank_dict[move[0]], int(move[1])-1
    
    return [r, f, d, ambig]

In [9]:
parse_move("Kxa4")

[7, 3, 5, None]

In [10]:
board = init_board()

In [12]:
def unroll(board):
    return board.ravel()

In [13]:
unroll(board)

array([ 0, -1,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0, -1,  0,
        0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0, -1,
        0,  0,  0,  0, -1,  0,  0,  0,  0, -1,  0,  0,  0,  0, -1,  0,  0,
        0,  0,  0, -1,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0, -1,  0,
        0,  0,  0,  0, -1,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0, -1,
        0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0

In [14]:
def move_R(to, player):
    r, f, d, ambig = to
   
    if np.array([board[:, :, d] == player]).sum() == 1:
        return move_simple(to, player)
        
    elif np.array([board[:, f, d] == player]).sum() == 1:
        board[:, f, d] = np.multiply(board[:, f, d],
                                     np.array([board[:, f, d] != player]))
        
    elif np.array([board[r, :, d] == player]).sum() == 1:
        board[r, :, d] = np.multiply(board[r, :, d],
                                     np.array([board[r, :, d] != player]))
                
    else:
        try:
            a = int(ambig)-1
            board[:, a, d] = np.multiply(board[:, a, d],
                                         np.array([board[:, a, d] != player]))
            
        except ValueError:
            rank_dict = {"a": 7, "b": 6, "c": 5, "d": 4, "e": 3, "f": 2, "g": 1, "h": 0}
            a = rank_dict(ambig)
            board[a, :, d] = np.multiply(board[a, :, d],
                                         np.array([board[a, :, d] != player]))
        
    board[r, f, d] = player
        

In [15]:
board = init_board()
move = parse_move("R8a5")
print(move)
move_R(move, 1)
board[:, :, 1]

[7, 4, 1, '8']


array([[-1,  0,  0,  0,  0,  0,  0, -1],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  0,  0,  0,  1,  0,  0,  0]], dtype=int8)

In [16]:
[list(range(0, 3)), 10]

[[0, 1, 2], 10]

In [25]:
r = ["a", "b", "c", "d", "e", "f", "g", "h"]
f = list(range(1, 9))

move = "e4"

In [51]:
def find_diags(to):
    """ Finds squares on both diagonals that target square lies on """
    r, f, d, ambig = to
    
    diags = set()
    
    # Down-left    
    r_, f_ = r, f
    while r_ >= 0 and f_ <= 7:
        diags.add((r_, f_))
        r_ -= 1
        f_ += 1
    # Up-right
    r_, f_ = r, f
    while f_ >= 0 and r_ <= 7:
        diags.add((r_, f_))
        r_ += 1
        f_ -= 1    
    # Down-right
    r_, f_ = r, f
    while max(r_, f_) <= 7:
        diags.add((r_, f_))
        r_ += 1
        f_ += 1
    # Up-left
    r_, f_ = r, f
    while min(r_, f_) >= 0:
        diags.add((r_, f_))
        r_ -= 1
        f_ -= 1  
        
    return list(diags)

In [52]:
find_diags([3, 5, 2, None])

[(3, 5),
 (2, 6),
 (1, 3),
 (4, 6),
 (7, 1),
 (0, 2),
 (4, 4),
 (5, 7),
 (6, 2),
 (1, 7),
 (2, 4),
 (5, 3)]

In [53]:
def move_B(to, player):
    r, f, d, ambig = to
   
    # If there's only one piece of that type
    if np.array([board[:, :, d] == player]).sum() == 1:
        return move_simple(to, player)
    
    # Only one bishop on each square colour; so just wipe both diagonals        
    else:
        diags = find_diags(to)
        for square in diags:
            r_, f_ = square
            if board[r_, f_, 3] == player:
                board[r_, f_, 3] = 0
        
    board[r, f, d] = player

In [59]:
board = init_board()
board[:, :, 3]

array([[ 0,  0, -1,  0,  0, -1,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  1,  0,  0,  1,  0,  0]], dtype=int8)

In [62]:
move_B(parse_move("Bf8"), 1)
board[:, :, 3]
move_B(parse_move("Bg2"), -1)
board[:, :, 3]

array([[ 0,  0,  0,  0,  0, -1,  0,  0],
       [ 0, -1,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  1,  0,  0]], dtype=int8)